subprocess 模块提供了了三个 API 处理进程:
- Python 3.5 中添加的 run() 函数，是一个运行进程的 API，也可以收集其输出
- call()，check_call() 以及 check_output() 是从 Python2 继承的较早的高级 API。在已存的程序中，它们仍然被广泛支持和使用
- 类 Popen 是一个低级 API，用于构建其他的 API 以及用于更复杂的进程交互。Popen 构造函数接受参数设置新进程，以便父进程可以通过管道与它通信。

## 运行外部命令

In [2]:
import subprocess

completed = subprocess.run(['ls', '-l'])  # 如果在python下是能输出结果的，但在jupyter中只能按下例打印stdout
print('return code:',completed.returncode)

return code: 0


In [13]:
completed = subprocess.run('echo $HOME',stdout=subprocess.PIPE, shell=True)
# 设置 shell 参数为 True 会导致 subprocess 创建一个新的中间 shell 进程运行命令。默认的行为是直接运行命令
print('returncode:', completed.returncode)
print(completed.stdout)

returncode: 0
b'/Users/hejl\n'


In [11]:
try:
    subprocess.run(['false',], check=True)
except subprocess.CalledProcessError as err:
    print('ERROR:', err)

ERROR: Command '['false']' returned non-zero exit status 1.


In [3]:
try:
    subprocess.run(['false',])
except subprocess.CalledProcessError as err:
    print('ERROR:', err)

In [6]:
try:
    completed = subprocess.run(
        'echo to stdout; echo to stderr 1>&2; exit 1',
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
except subprocess.CalledProcessError as err:
    print('ERROR:', err)
else:
    print('returncode:', completed.returncode)
    print('Have {} bytes in stdout: {!r}'.format(
        len(completed.stdout),
        completed.stdout.decode('utf-8'))
    )
    print('Have {} bytes in stderr: {!r}'.format(
        len(completed.stderr),
        completed.stderr.decode('utf-8'))
    )

returncode: 1
Have 10 bytes in stdout: 'to stdout\n'
Have 10 bytes in stderr: 'to stderr\n'


某些情况下，输出不应该被展示和捕获，使用 DEVNULL 抑制输出流。

In [7]:
try:
    completed = subprocess.run(
        'echo to stdout; echo to stderr 1>&2; exit 1',
        shell=True,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )
except subprocess.CalledProcessError as err:
    print('ERROR:', err)
else:
    print('returncode:', completed.returncode)
    print('stdout is {!r}'.format(completed.stdout))
    print('stderr is {!r}'.format(completed.stderr))

returncode: 1
stdout is None
stderr is None


## 使用管道
与进程单向通信

In [11]:
print('read:')
proc = subprocess.Popen(['echo', 'to stdout'], stdout=subprocess.PIPE)
value = proc.communicate()
print(value)
stdout_value = value[0].decode('utf-8')
print(stdout_value)


read:
(b'to stdout\n', None)
to stdout



In [15]:
print('write:')
proc = subprocess.Popen(['cat', '-'], stdin=subprocess.PIPE)
proc.communicate('stdin:sth'.encode('utf-8'))

write:


(None, None)

与进程双向通信

In [19]:
print('popen2:')
proc = subprocess.Popen('cat -; echo "to stderr" 1>&2',shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
msg = 'through stdin to stdout'.encode('utf-8')
stdout_value, stderr_value = proc.communicate(msg)
print(stdout_value)
print(stderr_value)

popen2:
b'through stdin to stdout'
b'to stderr\n'


In [20]:
print('popen3:')
proc = subprocess.Popen('cat -; echo "to stderr" 1>&2',shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
msg = 'through stdin to stdout'.encode('utf-8')
stdout_value, stderr_value = proc.communicate(msg)
print(stdout_value)
print(stderr_value)

popen3:
b'through stdin to stdoutto stderr\n'
None


连接管道

In [25]:
cat = subprocess.Popen(['cat', 'signal.ipynb'], stdout=subprocess.PIPE)
grep = subprocess.Popen(['grep', 'def'],stdin=cat.stdout, stdout=subprocess.PIPE)
cut = subprocess.Popen(['cut', '-b', '-30'],stdin=grep.stdout, stdout=subprocess.PIPE)
for line in cut.stdout:
    print(line)

b'    "def receive_signal(signum\n'
b'      "SIGINT     ( 2) : <buil\n'
b'    "def alarm_received(n, sta\n'
b'    "def receive_alarm(signum,\n'
b'    "def do_exit(sig, stack):\\\n'
b'    "def signal_handler(num, s\n'
b'    "def wait_for_signal():\\n"\n'
b'    "def send_signal():\\n",\n'
b'    "def signal_handler(num, s\n'
b'    "def use_alarm():\\n",\n'


相当于:  
```
$ cat signal.ipynb | grep "def" | cut -b -30
```

同另一个命令交互  
脚本 repeater.py 被用作下一个例子的子进程。它从 stdin 读取并且写入到 stdout ，一次一行，直到再没有输入。当开始和停止的时候，它也往 stderr 写入了一条消息，展示子进程的声明周期。

In [32]:
import io

print('one line at a time:')
proc = subprocess.Popen('python3 repeater.py',shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
stdin = io.TextIOWrapper(proc.stdin, encoding='utf-8', line_buffering=True)
stdout = io.TextIOWrapper(proc.stdout, encoding='utf-8',)
for i in range(5):
    line = f'{i} \n'
    stdin.write(line)
    output=stdout.readline()
    print(output)
    
remainder = proc.communicate()[0].decode('utf-8')
print(remainder)

print()
print('All line at a time:')
proc = subprocess.Popen('python3 repeater.py',shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
stdin = io.TextIOWrapper(proc.stdin, encoding='utf-8')
stdout = io.TextIOWrapper(proc.stdout, encoding='utf-8',)
for i in range(5):
    line = f'{i} \n'
    stdin.write(line)
stdin.flush()
    
remainder = proc.communicate()[0].decode('utf-8')
print(remainder)

one line at a time:
0 

1 

2 

3 

4 



All line at a time:
0 
1 
2 
3 
4 



## 进程间的信号

signal_parent.py & signal_child.py

subprocess_signal_parent_shell.py  & subprocess_signal_setpgrp.py